In [21]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [22]:
ces_bay = pd.read_csv('ces_bay_cleaned.csv')
ces_bay

,Census Tract,Total Population,California County,ZIP,Approximate Location,Longitude,Latitude,Ozone,PM2.5,Diesel PM,...,Low Birth Weight,Cardiovascular Disease,Education,Linguistic Isolation,Poverty,Unemployment,Housing Burden,Children < 10 years (%),Pop 10-64 years (%),Elderly > 64 years (%)
0,6095250701,3529,Solano,94590,Vallejo,-122.244902,38.090042,0.036,9.095673,0.411,...,6.420,21.220,24.7,17.60,34.60,12.7,25.2,12.52,75.180,12.300
1,6001407300,2420,Alameda,94601,Oakland,-122.210924,37.762179,0.029,9.082865,0.633,...,7.350,12.410,37.9,12.90,50.10,11.0,35.5,6.28,87.480,6.240
2,6001409000,4687,Alameda,94621,Oakland,-122.221368,37.720011,0.031,8.709957,0.562,...,8.290,15.260,31.0,13.70,44.70,7.3,23.0,18.33,72.970,8.700
3,6001408800,7149,Alameda,94621,Oakland,-122.196942,37.758804,0.031,8.834295,0.321,...,8.950,21.040,43.0,6.20,65.10,8.7,23.3,20.74,70.260,8.990
4,6013365002,5590,Contra Costa,94801,North Richmond,-122.384395,37.959466,0.032,9.145973,0.198,...,6.850,16.390,43.5,27.30,44.70,4.8,25.5,17.87,75.170,6.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,6081984300,0,San Mateo,94128,Unincorporated San Mateo County area,-122.376861,37.614758,0.034,8.789373,1.039,...,4.830,8.355,6.5,7.10,12.75,3.4,14.6,11.47,72.335,16.195
1478,6085504417,269,Santa Clara,95035,Milpitas,-121.852539,37.432128,0.040,8.064637,0.006,...,5.025,8.400,6.9,9.30,7.10,3.9,14.0,11.15,56.510,32.340
1479,6085511608,3302,Santa Clara,94305,Stanford,-122.177218,37.429257,0.034,8.115386,0.533,...,5.025,9.640,8.5,9.30,21.90,7.4,14.0,0.58,99.300,0.120
1480,6095253000,6720,Solano,95687,Vacaville,-121.978785,38.320656,0.043,7.993241,0.023,...,5.205,15.250,36.5,3.65,23.10,5.6,15.9,0.00,88.940,11.060


In [23]:
ces_bay.columns

Index(['Census Tract', 'Total Population', 'California County', 'ZIP',
       'Approximate Location', 'Longitude', 'Latitude', 'Ozone', 'PM2.5',
       'Diesel PM', 'Drinking Water', 'Lead', 'Pesticides', 'Tox. Release',
       'Traffic', 'Cleanup Sites', 'Groundwater Threats', 'Haz. Waste',
       'Imp. Water Bodies', 'Solid Waste', 'Pollution Burden', 'Asthma',
       'Low Birth Weight', 'Cardiovascular Disease', 'Education',
       'Linguistic Isolation', 'Poverty', 'Unemployment', 'Housing Burden',
       'Children < 10 years (%)', 'Pop 10-64 years (%)',
       'Elderly > 64 years (%)'],
      dtype='object')

In [25]:


# Define the columns for each category based on the assumptions
exposures_columns = ['Ozone', 'PM2.5','Diesel PM','Drinking Water','Lead', 'Pesticides', 'Tox. Release','Traffic', 'Cleanup Sites', 'Groundwater Threats', 'Haz. Waste','Imp. Water Bodies', 'Solid Waste', 'Pollution Burden']  # Pollution measures
environmental_effects_columns = ['Cardiovascular Disease']  # Health outcomes
sensitive_populations_columns = ['Children < 10 years (%)', 'Elderly > 64 years (%)']  # Demographics
socioeconomic_factors_columns = ['Education', 'Poverty', 'Unemployment', 'Housing Burden']  # Socio-economic factors

# Normalize the columns within each category
exposures = ces_bay[exposures_columns].mean(axis=1)
environmental_effects = ces_bay[environmental_effects_columns].mean(axis=1)
sensitive_populations = ces_bay[sensitive_populations_columns].mean(axis=1)
socioeconomic_factors = ces_bay[socioeconomic_factors_columns].mean(axis=1)

# Apply the weighting for Environmental Effects (half as much as Exposures)
pollution_burden = (exposures + environmental_effects * 0.5) / 1.5

# Calculate the Population Characteristics as the average of Sensitive Populations and Socioeconomic Factors
population_characteristics = (sensitive_populations + socioeconomic_factors) / 2

# Normalize the pollution_burden and population_characteristics to ensure they have the same scale before multiplication
pollution_burden_normalized = (pollution_burden - pollution_burden.min()) / (pollution_burden.max() - pollution_burden.min())
population_characteristics_normalized = (population_characteristics - population_characteristics.min()) / (population_characteristics.max() - population_characteristics.min())

# Calculate the overall CalEnviroScreen Score
calenviroscreen_score = pollution_burden_normalized * population_characteristics_normalized

# Add the score to the data for reference
data_with_calenviroscreen_score = ces_bay.assign(Score=calenviroscreen_score)

# Sort the data based on the CalEnviroScreen score to find the best counties
best_counties_calenviroscreen = data_with_calenviroscreen_score.sort_values(by='Score', ascending=False)

# Display the top 5 counties based on the CalEnviroScreen score
best_counties_calenviroscreen[['California County', 'Score']]

best_counties_calenviroscreen = best_counties_calenviroscreen.groupby('California County')['Score'].mean()

In [26]:
best_counties_calenviroscreen = best_counties_calenviroscreen * 10000

In [27]:
best_counties_calenviroscreen.sort_values()

California County
San Francisco    262.600266
San Mateo        294.311891
Alameda          312.136633
Santa Clara      329.134751
Napa             330.671881
Marin            372.632033
Solano           430.799924
Contra Costa     438.036018
Name: Score, dtype: float64

In [28]:
score = best_counties_calenviroscreen.astype(int)

In [29]:
score.to_csv("../Data_Mining_255/score.csv")

In [18]:
data_table = pd.read_html('C:/Users/prana/Documents/Masters/Data Mining/Project/Data_Mining_255/3dec/Report.html', skiprows=1, header=0)[0]


In [19]:
data_table

,Census Tract,Census Tract ID from 2010 Census
0,Total Population,2019 ACS population estimates in census tracts
1,California County,California county that the census tract falls ...
2,ZIP,Postal ZIP Code that the census tract falls wi...
3,Approximate Location,Approximate city where each census tract is lo...
4,Longitude,Longitude of the centroid of the census tract
5,Latitude,Latitude of the centroid of the census tract
6,Ozone,Amount of daily maximum 8 hour Ozone concentra...
7,PM2.5,Annual mean PM 2.5 concentrations
8,Diesel PM,Diesel PM emissions from on-road and non-road ...
9,Drinking Water,Drinking water contaminant index for selected ...
